In [ ]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, UpSampling2D, concatenate, GlobalAveragePooling2D, Dense
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, activations
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from datetime import datetime

# Set the input image size
IMAGE_SIZE = (224, 224)


# Set the paths to your training and testing/validation data
train_path = 'G:/Dataset-20230523T171034Z-001/Dataset/train'
test_path = 'G:/Dataset-20230523T171034Z-001/Dataset/test'

# Create the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE + (3,))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add the input layer
input_layer = Input(shape=IMAGE_SIZE + (3,), name="input")

# Obtain the output of the last layer in the base model
base_model_output = base_model(input_layer)

In [ ]:
# Conv1_1_64
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_layer)
# Conv1_2_64
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
# Block1 output for later concatenation
block1_conv2_output = x
# Batch Normalization1
x = BatchNormalization(name='bn1')(x)
# MaxPooling1
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
# Conv2_1_128
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
# Conv2_2_128
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
# Block2 output for later concatenation
block2_conv2_output = x
# Batch Normalization2
x = BatchNormalization(name='bn2')(x)
# MaxPooling2
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
# Conv3_1_256
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
# Conv3_2_256
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
# Conv3_3_256
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
# Block3 output for later concatenation
block3_conv3_output = x
# Batch Normalization3
x = BatchNormalization(name='bn3')(x)
# MaxPooling3
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
# Conv4_1_512
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
# Conv4_2_512
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
# Conv4_3_512
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
# Block4 output for later concatenation
block4_conv3_output = x
# Batch Normalization4
x = BatchNormalization(name='bn4')(x)
# MaxPooling4
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
# Conv5_1_512
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
# Conv5_2_512
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
# Conv5_3_512
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
# Block5 output for later concatenation
block5_conv3_output = x

# Downsample the tensors to a consistent shape
block1_conv2_downsampled = MaxPooling2D((4, 4))(block1_conv2_output)
block2_conv2_downsampled = MaxPooling2D((2, 2))(block2_conv2_output)
block3_conv3_downsampled = block3_conv3_output
block4_conv3_upsampled = UpSampling2D((2, 2))(block4_conv3_output)
block5_conv3_upsampled = UpSampling2D((4, 4))(block5_conv3_output)


# Perform depth concatenation on the downsampled tensors
x = concatenate([block1_conv2_downsampled, block2_conv2_downsampled, block3_conv3_downsampled, block4_conv3_upsampled, block5_conv3_upsampled], axis=-1)

# Batch Normalization5
x = BatchNormalization(name='bn5')(x)

# Conv6_512
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block6_conv1')(x)

# GAPooling6
x = GlobalAveragePooling2D(name='gap6')(x)

# FC-12
output = Dense(4, activation='softmax', name='fc12')(x)

# Finally, create the model
model = Model(inputs=input_layer, outputs=output)

# Print a summary of the model architecture
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input[0][0]']                  
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [ ]:
# Configure the optimizer
adam = optimizers.Adam()

# Compile the model with sparse categorical cross-entropy loss, Adam optimizer, and accuracy metric
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Set the paths to your training and testing/validation data
train_path = 'G:/Dataset-20230523T171034Z-001/Dataset/train'
test_path = 'G:/Dataset-20230523T171034Z-001/Dataset/test'

# Create a training data generator
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='sparse')  # Update class_mode to 'sparse'

# Create a testing/validation data generator
test_generator = datagen.flow_from_directory(
    test_path,
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='sparse')

# Modify the activation function for binary classification
model.layers[-1].activation = activations.sigmoid

# Define a checkpoint to save the best model during training
checkpoint = ModelCheckpoint(filepath='mymodel.h5', verbose=2, save_best_only=True)

# Create a list of callbacks, including the checkpoint
callbacks = [checkpoint]

# Record the start time
start = datetime.now()

# Calculate the number of training and validation steps
train_steps = len(train_generator)
validation_steps = len(test_generator)

# Train the model using the generated data
model_history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    callbacks=callbacks,
    verbose=2)  # Set verbose=1

# Calculate the training duration
duration = datetime.now() - start
print("Training completed in time:", duration)

Found 7058 images belonging to 4 classes.
Found 1918 images belonging to 4 classes.
Epoch 1/10

Epoch 1: val_loss improved from inf to 1.64317, saving model to mymodel.h5
221/221 - 3896s - loss: 1.8287 - accuracy: 0.5737 - val_loss: 1.6432 - val_accuracy: 0.4953 - 3896s/epoch - 18s/step
Epoch 2/10

Epoch 2: val_loss improved from 1.64317 to 1.12584, saving model to mymodel.h5
221/221 - 3888s - loss: 0.7276 - accuracy: 0.7328 - val_loss: 1.1258 - val_accuracy: 0.5448 - 3888s/epoch - 18s/step
Epoch 3/10

Epoch 3: val_loss improved from 1.12584 to 1.00307, saving model to mymodel.h5
221/221 - 3883s - loss: 0.5440 - accuracy: 0.7999 - val_loss: 1.0031 - val_accuracy: 0.6762 - 3883s/epoch - 18s/step
Epoch 4/10

Epoch 4: val_loss did not improve from 1.00307
221/221 - 3893s - loss: 0.4352 - accuracy: 0.8405 - val_loss: 1.7383 - val_accuracy: 0.6220 - 3893s/epoch - 18s/step
Epoch 5/10

Epoch 5: val_loss did not improve from 1.00307
221/221 - 3865s - loss: 0.4070 - accuracy: 0.8463 - val_loss:

In [ ]:
# Plot training & validation loss values
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('Improved CNN Model accuracy values')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: ignored